# Ejercicio Práctico Solución de Koalas

* pip install findspark
* pip install pyspark==2.4.4
* pip install koalas
* pip install plotly
* pip install nbformat>=4.2.0
* conda install -n bigdata ipykernel --update-deps --force-reinstall
* pip install matplotlib

In [1]:
import os, sys

# 1) Mismo intérprete Python para driver y ejecutores
os.environ["PYSPARK_PYTHON"] = sys.executable
os.environ["PYSPARK_DRIVER_PYTHON"] = sys.executable

# 2) Arrow y zona horaria (evita el warning)
os.environ["PYARROW_IGNORE_TIMEZONE"] = "1"

# 3) Fuerza loopback local (mitiga VPN/firewall/hostnames raros)
os.environ["SPARK_LOCAL_IP"] = "127.0.0.1"

from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
    .appName("PoS-stable")
    # Propaga el mismo Python al ejecutor
    .config("spark.executorEnv.PYSPARK_PYTHON", sys.executable)
    # Propaga la var de Arrow al ejecutor
    .config("spark.executorEnv.PYARROW_IGNORE_TIMEZONE", "1")
    # Enlaza y anuncia el driver en localhost (evita timeouts con VPN/Firewall)
    .config("spark.driver.bindAddress", "127.0.0.1")
    .config("spark.driver.host", "127.0.0.1")
    # (Opcional pero útil) fija timezone de Spark
    .config("spark.sql.session.timeZone", "UTC")
    .getOrCreate()
)

# Test mínimo: fuerza una operación Python worker
import pyspark.pandas as ps
ps.range(10).head(3)  # esto hace collect y valida el canal Py4J/Python worker



,id
0,0
1,1
2,2


In [2]:
import findspark
findspark.init()

import pandas as pd
import pyspark

In [3]:
import numpy as np
import pandas as pd
import pyspark.pandas as ks

### Funciones básicas

In [4]:
pser = pd.Series([1, 3, 5, np.nan, 6, 8], name="s")
kser = ks.from_pandas(pser)



In [5]:
kser.sort_index()

0    1.0
1    3.0
2    5.0
3    NaN
4    6.0
5    8.0
Name: s, dtype: float64

In [6]:
pdf = pd.DataFrame({'A': np.random.rand(5),
                    'B': np.random.rand(5)})


kdf = ks.from_pandas(pdf)

### Visualización de datos

In [7]:
kdf.describe()

,A,B
count,5.000000,5.000000
mean,0.538837,0.443986
std,0.230026,0.351166
min,0.292680,0.016024
25%,0.387201,0.190092
50%,0.466598,0.426172
75%,0.695181,0.776923
max,0.852524,0.810718


In [8]:
kdf.sort_values(by='B')

,A,B
3,0.292680,0.016024
1,0.387201,0.190092
2,0.695181,0.426172
0,0.466598,0.776923
4,0.852524,0.810718


In [9]:
kdf.transpose()

,0,1,2,3,4
A,0.466598,0.387201,0.695181,0.292680,0.852524
B,0.776923,0.190092,0.426172,0.016024,0.810718


### Selección

In [10]:
kdf[['A', 'B']]

,A,B
0,0.466598,0.776923
1,0.387201,0.190092
2,0.695181,0.426172
3,0.292680,0.016024
4,0.852524,0.810718


In [11]:
kdf.loc[1:2]

,A,B
1,0.387201,0.190092
2,0.695181,0.426172


In [12]:
kdf.iloc[:3, 1:2]

,B
0,0.776923
1,0.190092
2,0.426172


### Aplicando funciones de Python a Koalas

In [13]:
kdf.apply(np.cumsum)

c:\Users\osval\anaconda3\envs\ProcBigdata25B\lib\site-packages\pyspark\pandas\utils.py:1016: PandasAPIOnSparkAdviceWarning: If the type hints is not specified for `apply`, it is expensive to infer the data type internally.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


,A,B
0,0.466598,0.776923
1,0.853800,0.967016
2,1.548981,1.393188
3,1.841661,1.409213
4,2.694185,2.219931


In [14]:
kdf.apply(lambda x: x ** 2)

c:\Users\osval\anaconda3\envs\ProcBigdata25B\lib\site-packages\pyspark\pandas\utils.py:1016: PandasAPIOnSparkAdviceWarning: If the type hints is not specified for `apply`, it is expensive to infer the data type internally.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


,A,B
0,0.217714,0.603610
1,0.149925,0.036135
2,0.483276,0.181623
3,0.085662,0.000257
4,0.726798,0.657264


### Agrupando datos

In [15]:
kdf.groupby(['A', 'B']).sum()

c:\Users\osval\anaconda3\envs\ProcBigdata25B\lib\site-packages\pyspark\pandas\groupby.py:893: FutureWarning: Default value of `numeric_only` will be changed to `False` instead of `True` in 4.0.0.
  warnings.warn(


,
A,B
0.466598,0.776923
0.387201,0.190092
0.695181,0.426172
0.292680,0.016024
0.852524,0.810718


### Generando gráficos

In [16]:
# This is needed for visualizing plot on notebook
%matplotlib inline

In [17]:
speed = [0.1, 17.5, 40, 48, 52, 69, 88]
lifespan = [2, 8, 70, 1.5, 25, 12, 28]
index = ['snail', 'pig', 'elephant',
         'rabbit', 'giraffe', 'coyote', 'horse']
kdf = ks.DataFrame({'speed': speed,
                   'lifespan': lifespan}, index=index)
kdf.plot.barh()

In [18]:
kdf = ks.DataFrame({
    'sales': [3, 2, 3, 9, 10, 6, 3],
    'signups': [5, 5, 6, 12, 14, 13, 9],
    'visits': [20, 42, 28, 62, 81, 50, 90],
}, index=pd.date_range(start='2019/08/15', end='2020/03/09',
                       freq='M'))
kdf.plot.area()

### Utilizando SQL en Koalas

In [19]:
kdf = ks.DataFrame({'year': [1990, 1997, 2003, 2009, 2014],
                    'pig': [20, 18, 489, 675, 1776],
                    'horse': [4, 25, 281, 600, 1900]})

In [21]:
ks.sql("SELECT * FROM {kdf} WHERE pig > 100", kdf=kdf)


,year,pig,horse
0,2003,489,281
1,2009,675,600
2,2014,1776,1900


In [22]:
pdf = pd.DataFrame({'year': [1990, 1997, 2003, 2009, 2014],
                    'sheep': [22, 50, 121, 445, 791],
                    'chicken': [250, 326, 589, 1241, 2118]})

In [ ]:
result = ks.sql('''
    SELECT ks.pig, pd.chicken
    FROM {kdf} ks
    INNER JOIN {pdf} pd
    ON ks.year = pd.year
    ORDER BY ks.pig, pd.chicken
''', kdf=kdf, pdf=pdf)   

print(result)


    pig  chicken
0    18      326
1    20      250
2   489      589
3   675     1241
4  1776     2118


### Trabajando con PySpark

In [25]:
kdf = ks.DataFrame({'A': [1, 2, 3, 4, 5], 'B': [10, 20, 30, 40, 50]})
sdf = kdf.to_spark()
type(sdf)

c:\Users\osval\anaconda3\envs\ProcBigdata25B\lib\site-packages\pyspark\pandas\utils.py:1016: PandasAPIOnSparkAdviceWarning:

If `index_col` is not specified for `to_spark`, the existing index is lost when converting to Spark DataFrame.



pyspark.sql.dataframe.DataFrame

In [26]:
sdf.show()

+---+---+
|  A|  B|
+---+---+
|  1| 10|
|  2| 20|
|  3| 30|
|  4| 40|
|  5| 50|
+---+---+

